In [1]:
import random

import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)
from trl import RewardTrainer
     

In [2]:
random.seed(42)

In [3]:

hf_dataset = load_dataset(
    "argilla/dolly-curated-comparison-falcon-7b-instruct", split="train"
)

Generating train split:   0%|          | 0/7401 [00:00<?, ? examples/s]

In [4]:
df = hf_dataset.to_pandas()
df

,prompt,response-1,response-2,category,original_response,external_id
0,What is Depreciation,What is Depreciation – 10 Important Facts to K...,What is Depreciation on a Car?\nDepreciation i...,open_qa,Depreciation is the drop in value of an asset ...,518
1,What do you know about the city of Aberdeen in...,"Aberdeen, the city in which I've lived for the...","As an AI language model, I don't have personal...",open_qa,Aberdeen is a city located in the North East o...,351
2,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,information_extraction,Thunderstorm season in the United States and C...,10567
3,"When did Peloton IPO?\nOn September 26, 2019, ...",When did Peloton IPO?\nPeloton launched its in...,When did Peloton IPO?\nPeloton IPO'd on May 26...,closed_qa,Peloton became a public company via an initial...,12412
4,What is the best way to answer an interview qu...,The best way to answer an interview question m...,Some of the best ways to answer an interview q...,creative_writing,The first recommended step is to ask clarifyin...,2521
...,...,...,...,...,...,...
7396,How do i accept the change,How do i accept the change in my life\nAccepti...,I's a great opportunity to improve. The only t...,brainstorming,Embrace the change and see the difference,15010
7397,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,information_extraction,"Brazil, Botafogo-SP, Corinthians, Fiorentina",9970
7398,Without quoting directly from the text give me...,Without quoting directly from the text give me...,Without quoting directly from the text give me...,summarization,"Brendon Small is a stand-up comedian, Creator...",14205
7399,Is Killing is Sin ? Is it ture,Is Killing is Sin ? Is it ture?\nKilling can b...,Is Killing is Sin ? Is it ture?\nKilling is no...,brainstorming,Killing a human being should not be sin becaus...,11253


In [5]:
# List of response options
responses = ["response-1", "response-2"]


def get_chosen_and_not_chosen(responses):
    """
    Given a list of responses, randomly selects one and returns it along with the non-selected response.

    Args:
        responses (list): List of response options.

    Returns:
        tuple: A tuple containing the chosen response, non-chosen response, and the index of the chosen response.
    """
    chosen_id = random.randint(0, len(responses) - 1)
    not_chosen_id = 1 - chosen_id

    return responses[chosen_id], responses[not_chosen_id], chosen_id


# List to store rows of data
rows = []

# Iterate through the hf_dataset
for record in hf_dataset:
    chosen, not_chosen, chosen_id = get_chosen_and_not_chosen(responses)
    chosen_from_falcon, _, _ = get_chosen_and_not_chosen(responses)

    # Append a new row to the 'rows' list
    rows.append(
        {
            "instruction": record["prompt"],
            "chosen_response": record[chosen],
            "rejected_response": record[not_chosen],
        }
    )

In [6]:
prepared_dataset = Dataset.from_list(rows)
prepared_dataset.to_pandas()

,instruction,chosen_response,rejected_response
0,What is Depreciation,What is Depreciation – 10 Important Facts to K...,What is Depreciation on a Car?\nDepreciation i...
1,What do you know about the city of Aberdeen in...,"As an AI language model, I don't have personal...","Aberdeen, the city in which I've lived for the..."
2,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...,Describe thunderstorm season in the United Sta...
3,"When did Peloton IPO?\nOn September 26, 2019, ...",When did Peloton IPO?\nPeloton launched its in...,When did Peloton IPO?\nPeloton IPO'd on May 26...
4,What is the best way to answer an interview qu...,Some of the best ways to answer an interview q...,The best way to answer an interview question m...
...,...,...,...
7396,How do i accept the change,I's a great opportunity to improve. The only t...,How do i accept the change in my life\nAccepti...
7397,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...,Extract the teams that the footballer Sócrates...
7398,Without quoting directly from the text give me...,Without quoting directly from the text give me...,Without quoting directly from the text give me...
7399,Is Killing is Sin ? Is it ture,Is Killing is Sin ? Is it ture?\nKilling can b...,Is Killing is Sin ? Is it ture?\nKilling is no...


In [7]:
prepared_dataset

Dataset({
    features: ['instruction', 'chosen_response', 'rejected_response'],
    num_rows: 7401
})

In [8]:
prepared_dataset_mini = prepared_dataset.select(range(1000))

In [9]:
prepared_dataset_mini

Dataset({
    features: ['instruction', 'chosen_response', 'rejected_response'],
    num_rows: 1000
})

In [10]:
from transformers import (
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    AutoTokenizer,
)

# Load tokenizer for the "facebook/opt-350m" model
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

# Prepare quantization parameters
quantization_config = BitsAndBytesConfig(load_in_8bit=False, load_in_4bit=True)

# Initialize the sequence classification model
model = AutoModelForSequenceClassification.from_pretrained(
    "facebook/opt-350m",
    quantization_config=quantization_config,  # Apply the quantization configuration
    device_map={"": 0},  # Assign the model to device 0
    trust_remote_code=True,  # Trust remote code
    num_labels=1,  # Set the number of labels for classification (in this case, 1)
)

# Disable cache in model configuration
model.config.use_cache = False

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\lang\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--facebook--opt-350m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

c:\Users\user\anaconda3\envs\lang\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# If the tokenizer's pad_token is not set, use eos_token as pad_token and update model's pad_token_id
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id


# Define a formatting function for processing examples
def formatting_func(examples):
    kwargs = {
        "padding": "max_length",
        "truncation": True,
        "max_length": 512,
        "return_tensors": "pt",
    }

    # Prepend the instruction and a line break to the chosen_response and rejected_response fields.
    prompt_plus_chosen_response = (
        examples["instruction"] + "\n" + examples["chosen_response"]
    )
    prompt_plus_rejected_response = (
        examples["instruction"] + "\n" + examples["rejected_response"]
    )

    # Tokenize the modified fields.
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs)
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)

    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0],
        "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0],
        "attention_mask_rejected": tokens_rejected["attention_mask"][0],
    }


# Apply the formatting function to the prepared dataset
formatted_dataset = prepared_dataset_mini.map(formatting_func)

# Split the formatted dataset into training and testing sets
formatted_dataset = formatted_dataset.train_test_split()

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments
from peft import LoraConfig
from trl import RewardTrainer

# Prepare training parameters
training_args = TrainingArguments(
    output_dir="./train_logs",  # Output folder
    max_steps=100,  # Maximum number of training steps
    per_device_train_batch_size=4,  # Batch size per GPU for training
    gradient_accumulation_steps=1,  # Number of steps to accumulate gradients
    learning_rate=1.0e-4,  # Learning rate
    optim="adamw_torch",  # Optimizer
    save_steps=50,  # How often to save checkpoints
    logging_steps=10,  # How often to log training information
    # report_to="tensorboard",  # Reporting method (in this case, TensorBoard)
    remove_unused_columns=False,  # Whether to remove unused columns
    evaluation_strategy="steps",  # Evaluation strategy
    num_train_epochs=5,  # Number of training epochs
)

# Prepare PEFT parameters
peft_config = LoraConfig(
    r=16,  # Value of r
    lora_alpha=16,  # Value of lora_alpha
    bias="none",  # Bias setting
    task_type="SEQ_CLS",  # Task type (Sequence Classification)
    modules_to_save=["scores"],  # Modules to save
)

# Prepare RewardTrainer
trainer = RewardTrainer(
    model=model,  # The model for reinforcement learning
    tokenizer=tokenizer,  # The tokenizer for processing input data
    args=training_args,  # Training arguments
    train_dataset=formatted_dataset["train"],  # Training dataset
    eval_dataset=formatted_dataset["test"],  # Evaluation dataset
    peft_config=peft_config,  # PEFT configuration
    max_length=512,  # Maximum length of input
)

# Execute training
trainer.train()

# Save the pretrained reward model
trainer.model.save_pretrained("./reward_model")

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/100 [00:00<?, ?it/s]

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
c:\Users\user\anaconda3\envs\lang\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.9291, 'grad_norm': 13.680692672729492, 'learning_rate': 9e-05, 'epoch': 0.05}


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Which is bigger, a lion or a house cat?       │ Which is bigger, a lion or a house cat?      │ [0.4486, 0.5514] │
│ A lion is bigger as it is a species of big    │ A house cat is bigger than a lion. The       │                  │
│ cats while a house cat is a type of           │ average male lion can weigh about            │                  │
│ domesticated animal.                          │ 1,300-2,000 lbs, while the average house cat │                  │
│                                               │ can range from 8-15 lbs depending on the     │                  │
│                                               │ breed.                                       │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Why do we feel lighter in weight in a         │ Why do we feel lighter in weight in a        │ [0.1968, 0.8032] │
│ swimming pool?                                │ swimming pool?                               │                  │
│ The weight you feel lighter in a swimming     │ In a swimming pool, you might feel lighter   │                  │
│ pool due to buoyancy. When you enter a        │ due to buoyancy effects. When you push up    │                  │
│ swimming pool, the water surrounding you      │ against the bottom of a pool, the water is   │                  │
│ pushes you up against the surface of the      │ forced to move upwards and thus creates      │                  │
│ pool. This force is equal to your weight, so  │ upward forces on your body. This upward      │                  │
│ you feel heavier in the water. However, if    │ force can make you feel lighter, as it takes │                  │
│ you float on the surface of the water, you    │ some pressure off your feet and back, and    │                  │
│ can actually feel lighter. This is why you    │ can make you feel like you're swimming more  │                  │
│ can breathe more easily in water compared to  │ easily.                                      │                  │
│ out of water. Additionally, when you dive     │                                              │                  │
│ underwater, the pressure on your body is      │                                              │                  │
│ increased, which causes you to inhale more    │                                              │                  │
│ air and become more buoyant, resulting in a   │                                              │                  │
│ feeling of weightlessness.                    │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ If someone is tall, what sport should they    │ If someone is tall, what sport should they   │ [0.547, 0.453]   │
│ try?                                          │ try?                                         │                  │
│ If someone is tall, they may have a natural   │ It really depends on the person's            │                  │
│ advantage in sports like basketball,          │ preferences, but generally basketball and    │                  │
│ volleyball, and swimming. However, there are  │ volleyball are good choices for tall         │                  │
│ many other sports that can be beneficial      │ individuals. Other sports that could be      │                  │
│ based on their height, such as tennis or      │ suitable are swimming, soccer, and tennis.   │                  │
│ gymnastics.                                   │ It's a

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Which is bigger, a lion or a house cat?       │ Which is bigger, a lion or a house cat?      │ [0.3397, 0.6603] │
│ A lion is bigger as it is a species of big    │ A house cat is bigger than a lion. The       │                  │
│ cats while a house cat is a type of           │ average male lion can weigh about            │                  │
│ domesticated animal.                          │ 1,300-2,000 lbs, while the average house cat │                  │
│                                               │ can range from 8-15 lbs depending on the     │                  │
│                                               │ breed.                                       │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ Why do we feel lighter in weight in a         │ Why do we feel lighter in weight in a        │ [0.4033, 0.5967] │
│ swimming pool?                                │ swimming pool?                               │                  │
│ The weight you feel lighter in a swimming     │ In a swimming pool, you might feel lighter   │                  │
│ pool due to buoyancy. When you enter a        │ due to buoyancy effects. When you push up    │                  │
│ swimming pool, the water surrounding you      │ against the bottom of a pool, the water is   │                  │
│ pushes you up against the surface of the      │ forced to move upwards and thus creates      │                  │
│ pool. This force is equal to your weight, so  │ upward forces on your body. This upward      │                  │
│ you feel heavier in the water. However, if    │ force can make you feel lighter, as it takes │                  │
│ you float on the surface of the water, you    │ some pressure off your feet and back, and    │                  │
│ can actually feel lighter. This is why you    │ can make you feel like you're swimming more  │                  │
│ can breathe more easily in water compared to  │ easily.                                      │                  │
│ out of water. Additionally, when you dive     │                                              │                  │
│ underwater, the pressure on your body is      │                                              │                  │
│ increased, which causes you to inhale more    │                                              │                  │
│ air and become more buoyant, resulting in a   │                                              │                  │
│ feeling of weightlessness.                    │                                              │                  │
├───────────────────────────────────────────────┼──────────────────────────────────────────────┼──────────────────┤
│ If someone is tall, what sport should they    │ If someone is tall, what sport should they   │ [0.4644, 0.5356] │
│ try?                                          │ try?                                         │                  │
│ If someone is tall, they may have a natural   │ It really depends on the person's            │                  │
│ advantage in sports like basketball,          │ preferences, but generally basketball and    │                  │
│ volleyball, and swimming. However, there are  │ volleyball are good choices for tall         │                  │
│ many other sports that can be beneficial      │ individuals. Other sports that could be      │                  │
│ based on their height, such as tennis or      │ suitable are swimming, soccer, and tennis.   │                  │
│ gymnastics.                                   │ It's a

c:\Users\user\anaconda3\envs\lang\Lib\site-packages\transformers\tokenization_utils_base.py:2888: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\user\anaconda3\envs\lang\Lib\site-packages\trl\trainer\utils.py:657: UserWarning: There are 15 out of 250 instances where the predictions for both options are equal. As a consequence the accuracy can be misleading.
  warnings.warn(


{'eval_loss': 0.8033300638198853, 'eval_accuracy': 0.524, 'eval_runtime': 15.0616, 'eval_samples_per_second': 16.599, 'eval_steps_per_second': 2.125, 'epoch': 0.53}
{'train_runtime': 255.7765, 'train_samples_per_second': 1.564, 'train_steps_per_second': 0.391, 'train_loss': 0.8448475694656372, 'epoch': 0.53}


In [14]:
import torch


def get_score(model, tokenizer, prompt, response):
    """
    Computes a score for a given prompt and response using a provided model and tokenizer.

    Args:
        model (nn.Module): The model for scoring.
        tokenizer: The tokenizer for processing input data.
        prompt (str): The prompt text.
        response (str): The response text.

    Returns:
        float: The computed score.
    """
    # Tokenize the input sequences
    inputs = tokenizer.encode_plus(
        prompt,
        response,
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    ).to("cuda:0")

    # Perform forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the logits
    logits = outputs.logits

    return logits.item()

In [15]:
x = 4242
prepared_dataset[x]

{'instruction': 'Given this paragraph about WWII, how many fatalities happened?\n"World War II or the Second World War, often abbreviated as WWII or WW2, was a global conflict that lasted from 1939 to 1945. The vast majority of the world\'s countries, including all of the great powers, fought as part of two opposing military alliances: the Allies and the Axis. Many participants threw their economic, industrial, and scientific capabilities behind this total war, blurring the distinction between civilian and military resources. Aircraft played a major role, enabling the strategic bombing of population centres and the delivery of the only two nuclear weapons ever used in war.\n\nWorld War II was by far the deadliest conflict in history; it resulted in an estimated 70 to 85 million fatalities, mostly among civilians. Tens of millions died due to genocides (including the Holocaust), starvation, massacres, and disease. In the wake of the Axis defeat, Germany and Japan were occupied, and war 

In [16]:
# Get the prompt and responses for the example
prompt = prepared_dataset[x]["instruction"]
rejected_response = prepared_dataset[x]["rejected_response"]
chosen_response = prepared_dataset[x]["chosen_response"]

# Get the score for the example with the less preferred response
score_less_pref = get_score(model, tokenizer, prompt, rejected_response)
print(f"Score for less preferred response: {score_less_pref}")

# Get the score for the example with the preferred response
score_pref = get_score(model, tokenizer, prompt, chosen_response)
print(f"Score for preferred response: {score_pref}")

Score for less preferred response: -3.486865282058716
Score for preferred response: -3.5882911682128906
